In [1]:
# ////////////////////
# // This script combines multiple Good Fire steps and allows it to be summarized and exported easily for various datasets
# // The earliest year that this script can analyze is 1986, as the first LCMS/LCMAP year is 1985
# // This script incorporates reburn logic and manages FRG5

# // It does the following:
# // -reclassifies the Fire Regime Groups layer into low/mix & high/replacement
# // -generates an annual conservative forest mask for the year before each fire
# // -reads in bias corrected cbi (CBI_bc) generated annually in the year of each fire by script 'a01_generate_gf_cbi'
# // -exports summaries of the data for the summarizing features imported in "Summarizing Features"
# // -exports fire event-level summaries
# // -exports a flattened CBI layer for the entire period for a given AOI for use in visualizations



In [ ]:
import os
from functools import reduce

import ee
import geemap

from find_set_root import find_set_project_root
PROJECT_ROOT = find_set_project_root()
print(f"Project root found at: {PROJECT_ROOT}")
import utils.gee_functions as ugeef

ee.Initialize()

# USER-SET PARAMETERS
EXPORT_FOLDER = 'GEE_Exports'
VERSION = 'welty_wildfire'

# Load datasets
fires_raw = ee.FeatureCollection("projects/ee-tymc5571-goodfire/assets/welty_wildfire_1984_2020")
fire_year_attribute = 'Fire_Year'

gf_cbi = ee.Image("projects/ee-tymc5571-goodfire/assets/welty_wildfire_cbi_bc_1985_2020")


# Summarizing Features
states = ee.FeatureCollection('TIGER/2018/States')
western_states_names = [
    'Washington', 'Oregon', 'California', 'Idaho', 'Nevada', 'Montana',
    'Wyoming', 'Utah', 'Colorado', 'Arizona', 'New Mexico'
]

SUMMARIZE_FEATURES = states.filter(ee.Filter.inList('NAME', western_states_names))
SUMMARIZE_NAME = 'states'

VIZ_RASTER_AOI = states.filter(ee.Filter.eq('NAME', 'California')).geometry().bounds()
VIZ_RASTER_AOI_NAME = 'california'

# Reporting years
earliest_reporting_year = ee.Number(2010).getInfo()  
latest_reporting_year = ee.Number(2020).getInfo()  


# Set up exports
EXPORT_ALL_GF_TO_ASSET = False
EXPORT_ANNUAL_SUMMARIES = True
EXPORT_EVENT_STATISTICS = True

# # This line allows for exporting of specific annual summary years if there are failures on the GEE server side.
# # Comment this line of code out to run all.
# target_years = [2014, 2017]

c:\Users\tymc5571\AppData\Local\miniconda3\envs\macrosystems\lib\site-packages\geemap\conversion.py:23: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


Project root found at: C:\Users\tymc5571\dev\a-number-on-good-fire


In [3]:
##################################################
# SET UP AND PREP DATA
##################################################


# Print equivalent (for testing)
print('firesRaw:', fires_raw.limit(10).getInfo())
print('CBI:', gf_cbi.getInfo())

# Extract year bands
cbi_years = gf_cbi.bandNames()
years = cbi_years.map(lambda b: ee.Number.parse(ee.String(b).split('_').get(2)))
years_sorted = years.sort()
start_year = ee.Number(years_sorted.get(0)).getInfo()
end_year = ee.Number(years_sorted.get(-1)).getInfo()

print('Start year in CBI record:', start_year)
print('End year in CBI record:', end_year)

# Stable Script Data
lcms = ee.ImageCollection("USFS/GTAC/LCMS/v2022-8")
lcpri = ee.ImageCollection("projects/sat-io/open-datasets/LCMAP/LCPRI")
frg = ee.ImageCollection("LANDFIRE/Fire/FRG/v1_2_0")

# Generate unique years list
unique_years = ee.List.sequence(start_year, end_year).map(
    lambda year: ee.Number(year).format().slice(0, -2)
)
n_years = unique_years.length()

print('The CBI data contains data for', n_years.getInfo(), 'years. Those years are:', unique_years.getInfo())


# FIRE REGIME GROUP MASKS

# Select the CONUS image from the FRG image collection
frg_conus = frg.filterMetadata('system:index', 'contains', 'CONUS').first()

# Reclassify FRG values with descriptive mapping
frg_rcl = frg_conus.remap(
    [1, 2, 3, 4, 5, 111, 112, 131, 132, 133],
    [1, 2, 3, 4, 5, 6, 6, 6, 6, 6],
    0,
    'FRG'
).rename('frcRcls')

# Create masks for specific FRG categories
frg_low_mix_short = frg_rcl.eq(1).selfMask().rename('frgLowMixShort')
frg_replace_short = frg_rcl.eq(2).selfMask().rename('frgReplaceShort')
frg_low_mix_long = frg_rcl.eq(3).selfMask().rename('frgLowMixLong')
frg_replace_long = frg_rcl.eq(4).selfMask().rename('frgReplaceLong')
frg_any_200 = frg_rcl.eq(5).selfMask().rename('frgAny200')

# Combine short/long low-mix and replacement types
frg_low_mix = ee.ImageCollection([frg_low_mix_short.rename('frgLowMix'), frg_low_mix_long.rename('frgLowMix')]).mosaic().selfMask().rename('frgLowMix')
frg_replace = ee.ImageCollection([frg_replace_short.rename('frgReplace'), frg_replace_long.rename('frgReplace')]).mosaic().selfMask().rename('frgReplace')

# Print projection for verification
print('FRG projection:', frg_rcl.projection().getInfo())

# MANAGE GF CBI FROM SCRIPT 1

# Get band names from the CBI image
band_names = gf_cbi.bandNames()
print('CBI band names:', band_names.getInfo())

# Create an ImageCollection from each band of the GF CBI image
def make_cbi_image(band_name):
    band_name = ee.String(band_name)
    year = ee.Number.parse(band_name.split('_').get(2))
    year = band_name.split('_').get(2)
    return gf_cbi.select([band_name]).rename('cbi_bc').set('year', year)

cbi_collection = ee.ImageCollection(band_names.map(make_cbi_image))

# Print to verify
print('CBI collection:', cbi_collection.aggregate_array('year').getInfo())
print('CBI projection:', cbi_collection.first().projection().getInfo())
print('CBI collection band names:', cbi_collection.first().bandNames().getInfo())


firesRaw: {'type': 'FeatureCollection', 'columns': {'Assigned_F': 'String', 'Circle_Fla': 'Integer', 'Circleness': 'Float', 'Exclude_Fr': 'String', 'Fire_Attri': 'String', 'Fire_Polyg': 'Integer', 'Fire_Year': 'Integer', 'GIS_Acres': 'Float', 'GIS_Hectar': 'Float', 'Listed_F_1': 'String', 'Listed_F_2': 'String', 'Listed_F_3': 'String', 'Listed_F_4': 'String', 'Listed_F_5': 'String', 'Listed_F_6': 'String', 'Listed_F_7': 'String', 'Listed_Fir': 'String', 'Listed_Map': 'String', 'Listed_Not': 'String', 'Listed_Rx_': 'String', 'OBJECTID': 'Float', 'Overlap_Wi': 'String', 'Prescribed': 'String', 'Processing': 'String', 'Shape_Area': 'Float', 'Shape_Leng': 'Float', 'Source_Dat': 'String', 'USGS_Assig': 'Float', 'Wildfire_N': 'String', 'Wildfire_a': 'String', 'system:index': 'String'}, 'version': 1752174898270859, 'id': 'projects/ee-tymc5571-goodfire/assets/welty_wildfire_1984_2020', 'properties': {'system:asset_size': 183557121}, 'features': [{'type': 'Feature', 'geometry': {'type': 'Polygo

c:\Users\tymc5571\AppData\Local\miniconda3\envs\macrosystems\lib\site-packages\ee\deprecation.py:207: DeprecationWarning: 

Attention required for USFS/GTAC/LCMS/v2022-8! You are using a deprecated asset.
To make sure your code keeps working, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/USFS_GTAC_LCMS_v2022-8

  warnings.warn(warning, category=DeprecationWarning)


The CBI data contains data for 36 years. Those years are: ['1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']
FRG projection: {'type': 'Projection', 'wkt': 'PROJCS["unnamed", \n  GEOGCS["NAD83", \n    DATUM["North_American_Datum_1983", \n      SPHEROID["GRS 1980", 6378137.0, 298.2572221010042, AUTHORITY["EPSG","7019"]], \n      TOWGS84[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], \n      AUTHORITY["EPSG","6269"]], \n    PRIMEM["Greenwich", 0.0], \n    UNIT["degree", 0.017453292519943295], \n    AXIS["Longitude", EAST], \n    AXIS["Latitude", NORTH], \n    AUTHORITY["EPSG","4269"]], \n  PROJECTION["Albers_Conic_Equal_Area"], \n  PARAMETER["central_meridian", -96.0], \n  PARAMETER["latitude_of_origin", 23.0], \n  PARAMETER["standard_parallel_1", 29.5], \n  PARAMET

In [4]:
# /////////////////////////////////////////////////////////////////////////////////
# /////////////////////////////////////////////////////////////////////////////////
# ///////////       GENERATE ANNUAL CONSERVATIVE FOREST MASK      /////////////////
# /////////////////////////////////////////////////////////////////////////////////
# /////////////////////////////////////////////////////////////////////////////////

# Print image collections to view if necessary
# Uncomment the following lines to print the collections
# print('LCPRI:', lcpri.getInfo())
# print('LCMS:', lcms.getInfo())

# Reclassify an image to binary (1 = forest)
def reclassify_image_binary(value):
    def inner(image):
        binary = image.eq(value).selfMask()
        return binary.copyProperties(image, ['system:time_start'])
    return inner

# AND operation between two image collections
def combine_collections_with_and(collection1, collection2, and_name):
    list1 = collection1.toList(collection1.size())
    list2 = collection2.toList(collection2.size())
    zipped = list1.zip(list2)

    def and_func(pair):
        image1 = ee.Image(ee.List(pair).get(0))
        image2 = ee.Image(ee.List(pair).get(1))
        result = image1.And(image2).rename(and_name)
        return result.copyProperties(image1, ['system:time_start'])

    return ee.ImageCollection(zipped.map(and_func))

# Add year property from system:time_start
def add_year_property(image):
    year = image.date().get('year')
    return image.set('year', year)

# Set forest start year (1 year before CBI data begins)
forest_start_year = ee.Number(start_year).subtract(1)

# Filter LCPRI and LCMS collections by date
lcpri_cover = lcpri.filterDate(str(forest_start_year.getInfo()), str(end_year))
lcms_cover = lcms.filter(ee.Filter.eq('study_area', 'CONUS')) \
                 .filterDate(str(forest_start_year.getInfo()), str(end_year)) \
                 .select('Land_Cover')

# Reclassify forest (LCPRI=4, LCMS=1)
lcpri_for = lcpri_cover.map(reclassify_image_binary(4))
lcms_for = lcms_cover.map(reclassify_image_binary(1))

# Combine LCPRI and LCMS forests
conservative_forest = combine_collections_with_and(lcpri_for, lcms_for, 'conservativeForest') \
                        .map(add_year_property)

print('Conservative forest:', conservative_forest.aggregate_array('year').getInfo())

# Required forest years (CBI years - 1)
required_forest_years = unique_years.map(lambda y: ee.Number.parse(y).subtract(1)).distinct().sort()
actual_forest_years = conservative_forest.aggregate_array('year').distinct().sort()

# Find missing forest years
missing_forest_years = required_forest_years.filter(
    ee.Filter.inList('item', actual_forest_years).Not()
)
print('Missing conservative forest years:', missing_forest_years.getInfo())

# Determine available years in LCPRI and LCMS
lcpri_years = lcpri.aggregate_array('system:time_start') \
    .map(lambda d: ee.Date(d).get('year')).distinct()
lcms_years = lcms.aggregate_array('system:time_start') \
    .map(lambda d: ee.Date(d).get('year')).distinct()

available_from_lcpri = missing_forest_years.filter(ee.Filter.inList('item', lcpri_years))
available_from_lcms = missing_forest_years.filter(ee.Filter.inList('item', lcms_years))
available_from_either = available_from_lcpri.cat(available_from_lcms).distinct()
print('Missing forest years available from LCMS or LCPRI:', available_from_either.getInfo())

print(available_from_either.size().getInfo())
if available_from_either.size().getInfo() == 0:
    print("No missing forest years available from LCMS or LCPRI.") 
else:
    print("Missing forest years available from LCMS or LCPRI:", available_from_either.getInfo())

    # Fill missing years from available sources
    reclass_lcms = reclassify_image_binary(1)
    reclass_lcpri = reclassify_image_binary(4)

    def fill_year(year):
        year = ee.Number(year)
        lcpri_img = lcpri.filter(ee.Filter.calendarRange(year, year, 'year')).first()
        lcms_img = lcms.filter(ee.Filter.eq('study_area', 'CONUS')) \
                    .filter(ee.Filter.calendarRange(year, year, 'year')) \
                    .select('Land_Cover') \
                    .first()
        filled_img = ee.Algorithms.If(
            lcpri_img,
            reclass_lcpri(ee.Image(lcpri_img)).set('year', year),
            ee.Algorithms.If(
                lcms_img,
                reclass_lcms(ee.Image(lcms_img)).set('year', year),
                None
            )
        )
        return filled_img

    filled_forest_images = ee.List(missing_forest_years.map(fill_year))
    filled_collection = ee.ImageCollection(filled_forest_images).filter(ee.Filter.notNull(['year']))
    conservative_forest = conservative_forest.merge(filled_collection)

    print('Conservative forest filled:', conservative_forest.aggregate_array('year').getInfo())

# Determine final available years
final_forest_years = conservative_forest.aggregate_array('year').distinct().sort()
min_forest_year = ee.Number(final_forest_years.reduce(ee.Reducer.min()))
max_forest_year = ee.Number(final_forest_years.reduce(ee.Reducer.max()))

# Determine usable CBI years
usable_start_year = min_forest_year.add(1)
usable_end_year = max_forest_year.add(1)

print(usable_start_year.getInfo())
print(usable_end_year.getInfo())

# Get available CBI years
cbi_years = (
    cbi_collection.aggregate_array('year')
    .map(lambda x: ee.Number.parse(x))
    .distinct()
    .sort()
)
cbi_min_year = ee.Number(cbi_years.reduce(ee.Reducer.min()))
cbi_max_year = ee.Number(cbi_years.reduce(ee.Reducer.max()))
print(cbi_years.getInfo())

# Intersect ranges
startYear = usable_start_year.max(cbi_min_year).getInfo()
endYear = usable_end_year.min(cbi_max_year).getInfo()

# Regenerate uniqueYears
uniqueYears = ee.List.sequence(startYear, endYear).map(lambda y: ee.Number(y).format().slice(0, -2))
nYears = ee.Number(uniqueYears.length())

# Notify
print(f"All data is available to process years {startYear} through {endYear}. This time frame will be used.")


Conservative forest: [1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]
Missing conservative forest years: [1984]
Missing forest years available from LCMS or LCPRI: []
0
No missing forest years available from LCMS or LCPRI.
1986
2020
[1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]
All data is available to process years 1986 through 2020. This time frame will be used.


In [5]:
# /////////////////////////////////////////////////////////////////////////////////
# /////////////////////////////////////////////////////////////////////////////////
# ///////////             GENERATE ANNUAL GF IMAGES                 ///////////////
# /////////////////////////////////////////////////////////////////////////////////
# /////////////////////////////////////////////////////////////////////////////////


# Define the function to generate Good Fire image per year
def generate_good_fire_image(this_year):
    this_year = ee.String(this_year) # note that this_year has been left in string format for consistency with prior versions of the codebase

    this_year_cbi = ee.Image(
        cbi_collection.filter(ee.Filter.eq('year', this_year)).first()
    ).select('cbi_bc')

    year_prior_forest = ee.Image(
        conservative_forest.filter(
            ee.Filter.eq('year', ee.Number.parse(this_year).subtract(1))
        ).first()
    ).rename('yearPriorForest')

    cbi_forest = this_year_cbi.updateMask(year_prior_forest)
    cbi_low_mod = cbi_forest.lt(2.25).And(cbi_forest.gte(0.1)).rename('cbiLower') \
        .multiply(ee.Image.pixelArea()).selfMask()
    cbi_high = cbi_forest.gte(2.25).rename('cbiHigh') \
        .multiply(ee.Image.pixelArea()).selfMask()
    cbi_any_burned = cbi_forest.gte(0.1).rename('cbiAnyBurned') \
        .multiply(ee.Image.pixelArea()).selfMask()
    cbi_unburned = cbi_forest.lt(0.1).And(cbi_forest.gte(0)) \
        .rename('cbiUnburned').multiply(ee.Image.pixelArea()).selfMask()

    cbi_collection2 = cbi_collection.map(
        lambda img: img.set('year', ee.Number.parse(img.get('year')).toInt())
    )
    prior_cbi_collection = cbi_collection2.filter(ee.Filter.lt('year', ee.Number.parse(this_year))).select('cbi_bc')

    prior_cbi_collection = cbi_collection2.filter(ee.Filter.lt('year', ee.Number.parse(this_year))).select('cbi_bc')

    # Generate the conditional image explicitly
    def compute_years_prior_cbi():
        mosaic = prior_cbi_collection.map(lambda img: img.gte(0.1)).mosaic().selfMask()
        return ee.Algorithms.If(
            prior_cbi_collection.size().gt(0),
            mosaic.unmask(0), #need to unmask to ensure that .Not() works correctly in the regime matching & FRI step
            ee.Image(0).selfMask().Not()
        )

    # Use ee.Image() AFTER the conditional is resolved
    years_prior_cbi = ee.Image(compute_years_prior_cbi())


    # Good fire regime & FRI matches
    frg1_good = cbi_low_mod.updateMask(frg_low_mix_short).rename('frg1Good')
    frg2_good = cbi_high.updateMask(frg_replace_short).rename('frg2Good')
    frg3_good = cbi_low_mod.updateMask(frg_low_mix_long).updateMask(years_prior_cbi.Not()).rename('frg3Good')
    frg4_good = cbi_high.updateMask(frg_replace_long).updateMask(years_prior_cbi.Not()).rename('frg4Good')
    frg5_good = cbi_any_burned.updateMask(frg_any_200).updateMask(years_prior_cbi.Not()).rename('frg5Good')
    frg5_good_low_mod = frg5_good.updateMask(cbi_low_mod).rename('frg5GoodLowMod')
    frg5_good_high = frg5_good.updateMask(cbi_high).rename('frg5GoodHigh')

    # Too frequent
    too_frequent_low = cbi_low_mod.updateMask(frg_low_mix_long).updateMask(years_prior_cbi).rename('tooFrequentLow')
    too_frequent_high = cbi_high.updateMask(frg_replace_long).updateMask(years_prior_cbi).rename('tooFrequentHigh')
    too_frequent_any = cbi_any_burned.updateMask(frg_any_200).updateMask(years_prior_cbi).rename('tooFrequentAny200')

    # Not matching regime or unburned
    frg_low_cbi_high = cbi_high.updateMask(frg_low_mix).rename('lowerRegimeCbiHigh')
    frg_replace_cbi_low = cbi_low_mod.updateMask(frg_replace).rename('replaceRegimeCbiLow')
    frg_low_cbi_unburned = cbi_unburned.updateMask(frg_low_mix).rename('lowerRegimeCbiUnburned')
    frg_replace_cbi_unburned = cbi_unburned.updateMask(frg_replace).rename('replaceRegimeCbiUnburned')

    # Summary layers - NOT WORKING CORRECTLY
    lower_good_fire = ee.ImageCollection([frg1_good.rename('lowerGoodFire'), frg3_good.rename('lowerGoodFire'), frg5_good_low_mod.rename('lowerGoodFire')]).mosaic().rename('lowerGoodFire')
    high_good_fire = ee.ImageCollection([frg2_good.rename('highGoodFire'), frg4_good.rename('highGoodFire'), frg5_good_high.rename('highGoodFire')]).mosaic().rename('highGoodFire')
    good_fire_all = ee.ImageCollection([lower_good_fire.rename('goodFireAll'), high_good_fire.rename('goodFireAll')]).mosaic().rename('goodFireAll')

    unmatched_regime = ee.ImageCollection([frg_low_cbi_high.rename('unmatchedRegime'), frg_replace_cbi_low.rename('unmatchedRegime')]).mosaic().rename('unmatchedRegime')
    too_frequent = ee.ImageCollection([too_frequent_low.rename('tooFrequent'), too_frequent_high.rename('tooFrequent'), too_frequent_any.rename('tooFrequent')]).mosaic().rename('tooFrequent')

    forest_area = year_prior_forest.multiply(ee.Image.pixelArea()).selfMask()
    tot_area = ee.Image.pixelArea().rename('totalArea')

    all_gf = (
        frg1_good
        .addBands(frg2_good)
        .addBands(frg3_good)
        .addBands(frg4_good)
        .addBands(frg5_good)
        .addBands(frg5_good_low_mod)
        .addBands(frg5_good_high)
        .addBands(too_frequent_low)
        .addBands(too_frequent_high)
        .addBands(too_frequent_any)
        .addBands(frg_low_cbi_high)
        .addBands(frg_replace_cbi_low)
        .addBands(frg_low_cbi_unburned)
        .addBands(frg_replace_cbi_unburned)
        .addBands(lower_good_fire)
        .addBands(high_good_fire)
        .addBands(good_fire_all)
        .addBands(unmatched_regime)
        .addBands(too_frequent)
        .addBands(forest_area)
        .addBands(tot_area)
        .set('year', this_year)
    )

    return all_gf

# Map over unique years to generate the collection
all_gf = ee.ImageCollection(uniqueYears.map(generate_good_fire_image))

print('AllGF:', all_gf.aggregate_array('year').getInfo())

all_gf_with_year_num = all_gf.map(
    lambda img: img.set('year_num', ee.Number.parse(img.get('year')))
)
all_gf_reporting_years = all_gf_with_year_num.filter(
    ee.Filter.rangeContains('year_num', earliest_reporting_year, latest_reporting_year)
).sort('year_num')


print('AllGF reporting:', all_gf_reporting_years.aggregate_array('year').getInfo())


AllGF: ['1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']
AllGF reporting: ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']


In [6]:
# /////////////////////////////////////////////////////////////////////////////////
# /////////////////////////////////////////////////////////////////////////////////
# ///////////    GENERATE & EXPORT FLATTENED IMAGE FOR REPORTING YEARS   ///////////////
# /////////////////////////////////////////////////////////////////////////////////
# /////////////////////////////////////////////////////////////////////////////////



# Mosaic over years and flatten into binary presence layers
lg_all_reporting_years = all_gf_reporting_years.select('lowerGoodFire').mosaic().gt(0).unmask(0)
hgf_all_reporting_years = all_gf_reporting_years.select('highGoodFire').mosaic().gt(0).unmask(0)
unmatched_regime_all_reporting_years = all_gf_reporting_years.select('unmatchedRegime').mosaic().gt(0).unmask(0)
too_frequent_all_reporting_years = all_gf_reporting_years.select('tooFrequent').mosaic().gt(0).unmask(0)

# Combine non-good fire types
not_gf_all_reporting_years = unmatched_regime_all_reporting_years.bitwiseOr(too_frequent_all_reporting_years)

# Assign values:
# 3 = not good fire, 2 = high good fire, 1 = low good fire
not_gf_all_reporting_years = not_gf_all_reporting_years.where(not_gf_all_reporting_years.eq(1), 3)
hgf_all_reporting_years = hgf_all_reporting_years.where(hgf_all_reporting_years.eq(1), 2)

# Combine the three into a final classified image
all_reporting_years_gf_flat = lg_all_reporting_years.max(hgf_all_reporting_years).max(not_gf_all_reporting_years).selfMask()

# Uncomment to visualize the flattened Good Fire raster
# Map = geemap.Map(center=[37.5, -119], zoom=6)
# Map.addLayer(all_reporting_years_gf_flat, {'min': 0, 'max': 3, 'palette': ['white', 'hotpink']}, 'all Good Fire')
# Map

In [7]:
# # Initialize the map

# TEST_YEAR = '2020'
# TEST_YEAR_PRIOR = 2019
# Map = geemap.Map(center=[40, -100], zoom=5)

# # Optional: add to geemap display
# # Map.addLayer(all_gf_flat, {'min': 0, 'max': 3, 'palette': ['white', 'springgreen', 'greenyellow', 'pink']}, 'GF Flat')


# # # Generation layers
# # Map.addLayer(frg_low_mix, {'min': 0, 'max': 1, 'palette': ['black', 'yellow']}, 'FRG LowMix')
# # Map.addLayer(frg_replace, {'min': 0, 'max': 1, 'palette': ['black', 'red']}, 'FRG High')
# # Map.addLayer(frg_any_200, {'min': 0, 'max': 1, 'palette': ['black', 'orange']}, 'FRG Any200')

# # # CBI palette
# # cbi_palette_class = ["#0000CD", "#6B8E23", "#FFFF00", "#FFA500", "#FF0000"]

# # # Mosaic of all CBI
# # Map.addLayer(
# #     cbi_collection.mosaic(),
# #     {'min': 0, 'max': 3, 'palette': cbi_palette_class},
# #     'CBI All Mosaic'
# # )

# # # Conservative forest (first image only)
# # Map.addLayer(
# #     conservative_forest.sort('year').filter(ee.Filter.eq('year', TEST_YEAR_PRIOR)),
# #     {'min': 0, 'max': 1, 'palette': ['black', 'limegreen']},
# #     'Conservative Forest year prior'
# # )

# # # year of CBI
# # Map.addLayer(
# #     cbi_collection.sort('year').filter(ee.Filter.eq('year', TEST_YEAR)),
# #     {'min': 0, 'max': 3, 'palette': cbi_palette_class},
# #     'CBI year'
# # )

# Map.addLayer(
#     all_gf.sort('year').filter(ee.Filter.eq('year', TEST_YEAR)).select('frg1Good'),
#     {'min': 0, 'max': 1, 'palette': ['white', 'hotpink']},
#     'FRG1 Good'
# )

# # Map.addLayer(
# #     all_gf.sort('year').filter(ee.Filter.eq('year', TEST_YEAR)).select('frg2Good'),
# #     {'min': 0, 'max': 1, 'palette': ['white', 'plum']},
# #     'FRG2 Good'
# # )

# Map.addLayer(
#     all_gf.sort('year').filter(ee.Filter.eq('year', TEST_YEAR)).select('frg3Good'),
#     {'min': 0, 'max': 1, 'palette': ['white', 'lightgreen']},
#     'FRG3 Good'
# )

# # Map.addLayer(
# #     all_gf.sort('year').filter(ee.Filter.eq('year', TEST_YEAR)).select('frg4Good'),
# #     {'min': 0, 'max': 1, 'palette': ['white', 'lightblue']},
# #     'FRG4 Good'
# # )

# Map.addLayer(
#     all_gf.sort('year').filter(ee.Filter.eq('year', TEST_YEAR)).select('frg5GoodLowMod'),
#     {'min': 0, 'max': 1, 'palette': ['white', 'hotpink']},
#     'FRG5 Good Low Mod'
# )
# Map.addLayer(
#     all_gf.sort('year').filter(ee.Filter.eq('year', TEST_YEAR)).select('frg5GoodHigh'),
#     {'min': 0, 'max': 1, 'palette': ['white', 'plum']},
#     'FRG5 Good High'
# )

# Map.addLayer(
#     all_gf.sort('year').filter(ee.Filter.eq('year', TEST_YEAR)).select('lowerGoodFire'),
#     {'min': 0, 'max': 1, 'palette': ['white', 'limegreen']},
#     'lower good fire'
# )

# # Display the map
# Map




In [ ]:
##########################
# EXPORTS
##########################

if(EXPORT_ALL_GF_TO_ASSET):

    # Export all GF data to asset for future use
    years = all_gf.aggregate_array('year').getInfo()

    # Loop over each image in the collection
    for year in years:
        image = all_gf.filter(ee.Filter.eq('year', year)).first()

        task = ee.batch.Export.image.toAsset(
            image=image,
            description=f'all_gf_with_fri_{VERSION}_{year}',
            assetId=f'projects/ee-tymc5571-goodfire/assets/all_gf_with_fri_{VERSION}_{year}',
            region=SUMMARIZE_FEATURES.geometry().bounds(),
            scale=30,
            crs='EPSG:5070',
            maxPixels=1e13
        )
        task.start()
        print(f'Started gf layer export for year {year}')

    # Export flat image for visualization in figure
    gf_flat_task = ee.batch.Export.image.toDrive(
        image=all_reporting_years_gf_flat,
        description=f'gf_data_flatraster_{VERSION}_{VIZ_RASTER_AOI_NAME}_{earliest_reporting_year}_{latest_reporting_year}',
        folder=EXPORT_FOLDER,
        fileNamePrefix=f'gf_data_flatraster_{VERSION}_{VIZ_RASTER_AOI_NAME}_{earliest_reporting_year}_{latest_reporting_year}',
        region=VIZ_RASTER_AOI,
        scale=30,
        crs='EPSG:5070',
        maxPixels=1e13
    )
    gf_flat_task.start()


# GENERATE AND EXPORT ANNUAL SUMMARIES

if EXPORT_ANNUAL_SUMMARIES:

    # Loop over each year
    for year in range(earliest_reporting_year, latest_reporting_year + 1):

        if 'target_years' in globals():
            if year not in target_years:
                continue

        year_str = ee.Number(year).format()
        
        # Get the image for this year
        img = all_gf_reporting_years.filter(ee.Filter.eq('year', year_str)).first()

        # Reduce image over each feature in summarizeFeatures
        def reduce_feature(feature):
            reduction = img.reduceRegion(
                reducer=ee.Reducer.sum(),
                geometry=feature.geometry(),
                scale=30,
                tileScale=16,
                maxPixels=1e11
            )
            return feature.set(reduction)\
                        .set('year', year)\
                        .set('units', 'm^2')\
                        .setGeometry(None)

        # Map over feature collection
        mapped_reduction = SUMMARIZE_FEATURES.map(reduce_feature)

        # Set up export task
        summary_task = ee.batch.Export.table.toDrive(
            collection=mapped_reduction,
            description=f'gf_data_{year}_{SUMMARIZE_NAME}_{VERSION}',
            fileNamePrefix=f'gf_data_{year}_{SUMMARIZE_NAME}_{VERSION}',
            folder=EXPORT_FOLDER,
            fileFormat='CSV'
        )
        summary_task.start()
        print(f'Started summary export for year {year}')


# Export fire event-level summaries

if EXPORT_EVENT_STATISTICS:
    
    # Filter fire events by year attribute
    fires_raw_filt = fires_raw.filter(
        ee.Filter.And(
            ee.Filter.gte(fire_year_attribute, earliest_reporting_year),
            ee.Filter.lte(fire_year_attribute, latest_reporting_year)
        )
    )

    # Define the per-fire reduction function
    def gf_reduce_feature(f):
        # Extract year from fire feature
        f_yr = ee.Number(f.get(fire_year_attribute)).format()
        
        # Get matching year image from all_gf
        yr_img = ee.Image(all_gf_reporting_years.filter(ee.Filter.eq('year', f_yr)).first())

        # Reduce over fire geometry
        reduction = yr_img.reduceRegion(
            reducer=ee.Reducer.sum(),
            geometry=f.geometry(),
            scale=30,
            tileScale=16,
            maxPixels=1e11
        )

        # Attach results and remove geometry for CSV
        return f.set(reduction).setGeometry(None)

    # Apply the reducer to each fire
    full_dats_fires = fires_raw_filt.map(gf_reduce_feature)

    # Export as CSV to Google Drive
    event_task = ee.batch.Export.table.toDrive(
        collection=full_dats_fires,
        description=f'gf_data_fire_events_{earliest_reporting_year}_{latest_reporting_year}_{VERSION}',
        fileNamePrefix=f'gf_data_fire_events_{earliest_reporting_year}_{latest_reporting_year}_{VERSION}',
        folder=EXPORT_FOLDER,
        fileFormat='CSV'
    )

    event_task.start()
    print('Started export of fire event data.')


Started summary export for year 2014
Started summary export for year 2017
Started export of fire event data.
